In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentAnalysisFeature
from azure.ai.documentintelligence.models._models import AnalyzeResult

import pandas as pd
import numpy as np
import functools
import threading
import json
import io
import re
import os
import uuid
from datetime import datetime

import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
key = os.environ.get('OCR_KEY')
endpoint = os.environ.get('OCR_ENDPOINT')
credential = AzureKeyCredential(key)
document_intelligence_client = DocumentIntelligenceClient(endpoint, credential, api_version='2023-10-31-preview')
doc_path ='../data/table-test-document.pdf'
query_fields = ['first_name', 'city', 'state']

with open(doc_path, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", 
        analyze_request=f,
        content_type="application/octet-stream", 
    )
r_cloud = poller.result()

# Read Model

In [4]:
service_url = 'http://localhost:5000/'

# Define the URL and file path
url = f"{service_url}formrecognizer/documentModels/prebuilt-read:syncAnalyze?api-version=2022-08-31"
file_path = "../data/table-test-document.pdf"

# Define headers
headers = {
    'accept': '*/*',
    'Content-Type': 'application/octet-stream',
}

# Open the file and send the POST request
with open(file_path, 'rb') as file:
    response = requests.post(url, headers=headers, data=file)

r_local = json.loads(response.text)
#r_local = r_local['analyzeResult']

# General Document

In [45]:
service_url = 'http://localhost:5000/'

# Define the URL and file path
url = f"{service_url}formrecognizer/documentModels/prebuilt-document:syncAnalyze?api-version=2022-08-31"
file_path = "../data/table-test-document.pdf"
form_path = '../data/patient_intake_form_sample.jpg'

# Define headers
headers = {
    'accept': '*/*',
    'Content-Type': 'application/octet-stream',
}

# Open the file and send the POST request
with open(form_path, 'rb') as file:
    response = requests.post(url, headers=headers, data=file)

r_local = json.loads(response.text)
r_local = r_local['analyzeResult']
print(response.text)


{"status":"succeeded","createdDateTime":"2024-02-06T13:59:03Z","lastUpdatedDateTime":"2024-02-06T13:59:07Z","analyzeResult":{"apiVersion":"2022-08-31","modelId":"prebuilt-document","stringIndexType":"textElements","content":"Patient Information\nFirst Name:\nALEJANDRO\nLast Name:\nROSALEZ\nSex:\nMarital Status:\nMARRIED\nAddress:\n123 ANY STREET\nEmail Address:\nCity:\nANYTOWN\nDate of Birth:\n10/10/1982\nState:\nCA\nZip Code:\n12345\nPhone:\n646-555-0111\nEmergency Contact 1:\nFirst Name:\nCARLOS\nLast Name:\nSALAZAR\nPhone:\n212-555-0150\nRelationship to Patient:\nBROTHER\nEmergency Contact 2:\nFirst Name:\nJANE\nLast Name:\nDOE\nPhone:\n650-555-0123\nRelationship to Patient:\nFRIEND\nDid you feel fever or feverish lately?\nXl Yes No\nAre you having shortness of breath?\nL Yes XNO\nDo you have a cough?\nİYes X No\nDid you experience loss of taste or smell?\nYes No\nWhere you in contact with any confirmed COVID-19 positive patients?\nXYes ONO\nDid you travel in the past 14 days to any

# Inspect Response

In [19]:
print(f'cloud keys: {r_cloud.keys()}')
print(f'local keys: {r_local.keys()}')

cloud keys: dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'paragraphs', 'styles', 'contentFormat'])
local keys: dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'paragraphs', 'styles', 'languages'])


In [31]:
print(f'cloud keys: {r_cloud["pages"][0].keys()}')
print(f'local keys: {r_local["pages"][0].keys()}')

print(f'cloud keys: {r_cloud["styles"]}')
print(f'local keys: {r_local["styles"]}')

cloud keys: dict_keys(['pageNumber', 'angle', 'width', 'height', 'unit', 'words', 'lines', 'spans'])
local keys: dict_keys(['pageNumber', 'angle', 'width', 'height', 'unit', 'words', 'lines', 'spans', 'kind'])
cloud keys: [{'confidence': 1, 'spans': [{'offset': 34, 'length': 6}, {'offset': 56, 'length': 20}, {'offset': 303, 'length': 32}, {'offset': 339, 'length': 2}, {'offset': 346, 'length': 3}, {'offset': 354, 'length': 3}, {'offset': 374, 'length': 5}, {'offset': 387, 'length': 6}, {'offset': 399, 'length': 4}, {'offset': 410, 'length': 5}, {'offset': 423, 'length': 6}, {'offset': 433, 'length': 2}, {'offset': 441, 'length': 4}, {'offset': 452, 'length': 5}, {'offset': 465, 'length': 6}, {'offset': 475, 'length': 2}, {'offset': 482, 'length': 3}, {'offset': 492, 'length': 5}, {'offset': 505, 'length': 9}, {'offset': 519, 'length': 3}, {'offset': 543, 'length': 6}, {'offset': 553, 'length': 2}, {'offset': 560, 'length': 3}, {'offset': 569, 'length': 4}, {'offset': 580, 'length': 5},

In [21]:
print(f'cloud keys: {r_cloud["pages"][0]["words"][0].keys()}')
print(f'local keys: {r_local["pages"][0]["words"][0].keys()}')

cloud keys: dict_keys(['content', 'polygon', 'confidence', 'span'])
local keys: dict_keys(['content', 'polygon', 'confidence', 'span'])


In [27]:
print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["polygon"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["polygon"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["content"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["content"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["confidence"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["confidence"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["span"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["span"]}')

cloud keys: [187, 15, 313, 15, 313, 34, 187, 33]
local keys: [186, 13, 315, 14, 314, 37, 186, 35]
cloud keys: HANDWRITING
local keys: HANDWRITING
cloud keys: 0.993
local keys: 0.991
cloud keys: {'offset': 0, 'length': 11}
local keys: {'offset': 0, 'length': 11}


# Test Parsing Method

In [89]:
text_granularity = 'paragraph'
model_id = 'prebuilt-read'

def parse_ocr_result(result) -> pd.DataFrame:
    text_granularity = 'PARAGRPAH'
    model_id = 'prebuilt-layout'
    parsed_result = None
    # azure doesn't provide results on page level natively
    level = text_granularity
    if (level.upper() == 'PAGE'):
        text_granularity = "LINE"
    else:
        text_granularity = level.upper()

    for page in result['pages']:
        try:
            contains_handwriting = result.styles[0].is_handwritten
        except:
            contains_handwriting = False

        ocr_data = []
        
        # to calculate the average confidence
        if text_granularity != "WORD":
            word_confidences = [word['confidence'] for word in page['words']]
            total_confidence = sum(word_confidences)
            total_words = len(word_confidences)
            average_confidence = total_confidence / total_words if total_words > 0 else 0
            
        # extraction of (natively provided) results 
        if text_granularity == "PARAGRPAH":
            for paragraph_idx, paragraph in enumerate(result['paragraphs']):
                x1, y1, x2, y2, x3, y3, x4, y4 = paragraph['boundingRegions'][0]['polygon']

                try: 
                    role = paragraph['role']
                except:
                    role = ''

                paragrpah_info = {
                    "page": paragraph['boundingRegions'][0]['pageNumber'],
                    "paragraph": paragraph_idx,
                    "text": paragraph['content'],
                    "role": role,
                    "bb_x1": x1,
                    "bb_y1": y1,
                    "bb_x2": x2,
                    "bb_y2": y2,
                    "bb_x3": x3,
                    "bb_y3": y3,
                    "bb_x4": x4,
                    "bb_y4": y4,
                    "offset": paragraph['spans'][0]['offset'],
                    "length": paragraph['spans'][0]['length'],
                    
                }
                
                ocr_data.append(paragrpah_info)

        elif text_granularity == "LINE":
            for line_idx, line in enumerate(page['lines']):
                x1, y1, x2, y2, x3, y3, x4, y4 = line['polygon']

                line_info = {
                    "page": page['pageNumber'],
                    "line": line_idx,
                    "text": line['content'],
                    "bb_x1": x1,
                    "bb_y1": y1,
                    "bb_x2": x2,
                    "bb_y2": y2,
                    "bb_x3": x3,
                    "bb_y3": y3,
                    "bb_x4": x4,
                    "bb_y4": y4,
                    "offset": line['spans'][0]['offset'],
                    "length": line['spans'][0]['length'],
                }
                
                ocr_data.append(line_info)

        elif text_granularity == "WORD":
            for word in page.words:
                x1, y1, x2, y2, x3, y3, x4, y4 = word['polygon']

                word_info = {
                    "page": page['pageNumber'],
                    "text": word['content'],
                    "confidence": word['confidence'],
                    "bb_x1": x1,
                    "bb_y1": y1,
                    "bb_x2": x2,
                    "bb_y2": y2,
                    "bb_x3": x3,
                    "bb_y3": y3,
                    "bb_x4": x4,
                    "bb_y4": y4,
                    "offset": word['span']['offset'],
                    "length": word['span']['length'],
                    }
                
                ocr_data.append(word_info)
        
        df = pd.DataFrame(ocr_data)

        # in case texts should be aggreagted on page level
        if level.upper() == "PAGE":
            ocr_data = []
            page_info = {
                    "page": page['pageNumber'],
                    "text": "\n ".join(df['text']),
                    "avg_confidence": average_confidence,
                    "contains_handwriting": contains_handwriting,
                    "bb_x1": df["bb_x1"].min(),
                    "bb_y1": df["bb_y1"].min(),
                    "bb_x2": df["bb_x2"].max(),
                    "bb_y2": df["bb_y2"].min(),
                    "bb_x3": df["bb_x3"].max(),
                    "bb_y3": df["bb_y3"].max(),
                    "bb_x4": df["bb_x4"].min(),
                    "bb_x4": df["bb_x4"].max(),
                    }
            ocr_data.append(page_info)
            
            df = pd.DataFrame(ocr_data)

    if model_id == 'prebuilt-read' and text_granularity.upper() == 'PARAGRAPH': # 'read' model doesn't provide semantic role, only 'layout' does
        parsed_result = parsed_result.drop(columns=['role'])

    return df


df = parse_ocr_result(r_cloud)
df.head()

,page,paragraph,text,role,bb_x1,bb_y1,bb_x2,bb_y2,bb_x3,bb_y3,bb_x4,bb_y4,offset,length
0,1,0,"Liebe Handschrift,",,89,41,339,36,340,78,90,83,0,18
1,1,1,ich mag Dich nicht besonders. Jeder Drittkläss...,,81,124,917,108,923,399,86,415,19,372
2,1,2,"Aber brauche ich Dich noch in einem Alltag, in...",,87,423,946,438,942,667,83,652,392,319
3,1,3,Du bildest unsere Persönlichkeit ab. Trotzdem ...,,86,696,934,701,932,1056,84,1051,712,432
4,1,4,"Schriftliche grüße ,",,94,1081,338,1087,337,1128,93,1122,1145,20


In [95]:
r_cloud['paragraphs'][3].keys()

dict_keys(['spans', 'boundingRegions', 'content'])

In [51]:
for page in r_cloud['pages']:

    word_confidences = [word['confidence'] for word in page.words]
    print(word_confidences)

[0.993, 0.997, 0.986, 0.865, 0.984, 0.992, 0.984, 0.996, 0.994, 0.918, 0.981, 0.951, 0.97, 0.992, 0.992, 0.993, 0.994, 0.991, 0.991, 0.994, 0.997, 0.998, 0.998, 0.994, 0.993, 0.993, 0.998, 0.993, 0.994, 0.993, 0.997, 0.962, 0.993, 0.992, 0.998, 0.994, 0.994, 0.999, 0.999, 0.994, 0.991, 0.994, 0.988, 0.977, 0.989, 0.994, 0.992, 0.995, 0.951, 0.991, 0.945, 0.998, 0.231, 0.994, 0.924, 0.99, 0.981, 0.995, 0.852, 0.994, 0.99, 0.975, 0.951, 0.997, 0.967, 0.994, 0.974, 0.993, 0.999, 0.991, 0.995, 0.993, 0.997, 0.993, 0.994, 0.998, 0.996, 0.998, 0.997, 0.995, 0.99, 0.994, 0.918, 0.924, 0.998, 0.984, 0.988, 0.844, 0.972, 0.994, 0.994, 0.999, 0.999, 0.998, 0.988, 0.981, 0.963, 0.997, 0.994, 0.878, 0.154, 0.335, 0.859, 0.891, 0.99, 0.993, 0.998, 0.992, 0.988, 0.999, 0.998, 0.976, 0.988, 0.989, 0.998, 0.994, 0.998, 0.998, 0.994, 0.993, 0.99, 0.991, 0.997, 0.993, 0.991, 0.993, 0.989, 0.994, 0.993, 0.994, 0.993, 0.99, 0.959, 0.993, 0.997, 0.993, 0.989, 0.975, 0.994, 0.998, 0.992, 0.988, 0.998, 0.991

# Key Value Parsing Method

In [48]:
def parse_ocr_result(result) -> pd.DataFrame:
        key_value_pairs = result['keyValuePairs']
        form_data = []

        for pair in key_value_pairs:
            page_number = pair['key']['boundingRegions'][0]['pageNumber']
            key = pair['key']['content']
            key_x1, key_y1, key_x2, key_y2, key_x3, key_y3, key_x4, key_y4 = pair['key']['boundingRegions'][0]['polygon']
            key_offset = pair['key']['spans'][0]['offset']
            key_length = pair['key']['spans'][0]['length']

            try:
                value = pair['value']['content']
                value_x1, value_y1, value_x2, value_y2, value_x3, value_y3, value_x4, value_y4 = pair['value']['boundingRegions'][0]['polygon']
                value_offset = pair['value']['spans'][0]['offset']
                value_length = pair['value']['spans'][0]['length']
            except KeyError as e:
                value_x1 = value_y1 = value_x2 = value_y2 = value_x3 = value_y3 = value_x4 = value_y4 = None
                value_offset = value_length = None
                value = None

            key_value = {
                'page_number': page_number,
                'key': key,
                'value': value,
                'key_x1': key_x1,
                'key_y1': key_y1,
                'key_x2': key_x2,
                'key_y2': key_y2,
                'key_x3': key_x3,
                'key_y3': key_y3,
                'key_x4': key_x4,
                'key_y4': key_y4,
                'key_offset': key_offset,
                'key_length': key_length,
                'value_x1': value_x1,
                'value_y1': value_y1,
                'value_x2': value_x2,
                'value_y2': value_y2,
                'value_x3': value_x3,
                'value_y3': value_y3,
                'value_x4': value_x4,
                'value_y4': value_y4,
                'value_offset': value_offset,
                'value_length': value_length,
            }

            form_data.append(key_value)
        
        df = pd.DataFrame(form_data)

        return df

df = parse_ocr_result(r_local)
df.head()

,page_number,key,value,key_x1,key_y1,key_x2,key_y2,key_x3,key_y3,key_x4,...,value_x1,value_y1,value_x2,value_y2,value_x3,value_y3,value_x4,value_y4,value_offset,value_length
0,1,First Name:,ALEJANDRO,75,420,331,425,330,474,74,...,415.0,343.0,1049.0,366.0,1050.0,479.0,414.0,463.0,32.0,9.0
1,1,Last Name:,ROSALEZ,1110,422,1361,427,1360,478,1109,...,1384.0,355.0,1874.0,353.0,1874.0,482.0,1384.0,474.0,53.0,7.0
2,1,Date of Birth:,10/10/1982,2110,428,2419,428,2419,474,2110,...,2471.0,375.0,2935.0,374.0,2936.0,487.0,2472.0,479.0,158.0,10.0
3,1,Sex:,None,78,566,178,569,177,612,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Marital Status:,MARRIED,496,563,823,572,822,623,495,...,888.0,524.0,1239.0,531.0,1239.0,608.0,887.0,602.0,82.0,7.0


# Query Parsing

In [34]:
r_cloud['documents'][0]['fields']['city']['spans'][0]['offset']

[{'offset': 161, 'length': 7}]

# Table Parsing

In [37]:
def map_parsing(result) -> pd.DataFrame:
        tables = []
        table_output_format = 'map'
        select_table = True
        table_selection_method = 'index'
        table_selection_idx = 0

        # extract all table data
        for index, table in enumerate(result['tables']):
            if table_output_format.upper() == 'MAP':

                if not isinstance(table, dict):
                    table = table.as_dict()


                #dict = table.as_dict()
                df = pd.DataFrame.from_dict(table['cells'])

                # extract page_number and polygon coordinates
                df['page'] = df['boundingRegions'].apply(lambda x: x[0]['pageNumber'])
                df['table_index'] = index
                df['polygon'] = df['boundingRegions'].apply(lambda x: x[0]['polygon'])

                # extract polygon coordinates
                df['x1'] = df['polygon'].apply(lambda x: x[0])
                df['y1'] = df['polygon'].apply(lambda x: x[1])
                df['x2'] = df['polygon'].apply(lambda x: x[2])
                df['y2'] = df['polygon'].apply(lambda x: x[3])
                df['x3'] = df['polygon'].apply(lambda x: x[4])
                df['y3'] = df['polygon'].apply(lambda x: x[5])
                df['x4'] = df['polygon'].apply(lambda x: x[6])
                df['y4'] = df['polygon'].apply(lambda x: x[7])

                # extract offset and length
                df['offset'] = df['spans'].apply(lambda x: int(x[0]['offset']) if x else None)
                df['length'] = df['spans'].apply(lambda x: int(x[0]['length']) if x else None)

                # drop unnecessary columns
                df.drop(columns=['boundingRegions','spans', 'polygon'], inplace=True)

                table_info = {
                    'table_index': index,
                    'row_count': table['rowCount'],
                    'column_count': table['columnCount'],
                    'cell_count': table['rowCount']*table['columnCount'],
                    'table': df
                }

                tables.append(table_info)

        # select specific table (optional)
        if select_table:
            if table_selection_method.upper() == 'INDEX':
                parsed_result = tables[table_selection_idx]['table']
            elif table_selection_method.upper() == 'SIZE':
                # Find the entry with the highest cell_count using max function
                table_most_cells = max(tables, key=lambda x: x['cell_count'], default=None)
                parsed_result = table_most_cells['table'] if table_most_cells else None

        else:
            # combine all extracted tables (only works for output type 'map')
            parsed_result = pd.concat([table['table'] for table in tables], ignore_index=True)

        return parsed_result

df = map_parsing(r_local)
df.head()

,kind,rowIndex,columnIndex,content,page,table_index,x1,y1,x2,y2,x3,y3,x4,y4,offset,length
0,columnHeader,0,0,name,1,0,0.9982,3.2621,1.8992,3.2621,1.8992,3.492,0.9982,3.492,619.0,4.0
1,columnHeader,0,1,email,1,0,1.8992,3.2621,3.6390,3.2621,3.6390,3.492,1.8992,3.492,624.0,5.0
2,columnHeader,0,2,phone,1,0,3.6390,3.2621,5.1240,3.2621,5.1240,3.492,3.6390,3.492,630.0,5.0
3,columnHeader,0,3,birthdate,1,0,5.1240,3.2621,6.0188,3.2621,6.0188,3.492,5.1240,3.492,636.0,9.0
4,columnHeader,0,4,married,1,0,6.0188,3.2621,6.9260,3.2621,6.9260,3.492,6.0188,3.492,646.0,7.0


In [39]:
def result_to_dfs(result) -> list:
        tables = []
        for table in result['tables']:
            table_df = pd.DataFrame(columns=range(table['columnCount']), index=range(table['rowCount']))

            for cell in table['cells']:
                table_df.iloc[cell['rowIndex'], cell['columnIndex']] = cell['content']

            # use the first row as column names
            table_df.columns = table_df.iloc[0]
            table_df = table_df[1:]
            
            tables.append(table_df)
        return tables

def reference_parsing(result) -> pd.DataFrame: # TODO
        tables = result_to_dfs(result)
        table_output_library = 'test'
        table_info = []

        for table in tables:
            reference = uuid.uuid4()
            reference = re.sub(r'^\w{3}', 'tbl_', str(reference))
            reference = reference.replace('-', '')

            # save table to caslib
            try: 
                print(f'Save table {reference} to caslib {table_output_library}')
            except Exception as e:
                print(f'Failed to save table {reference} to caslib {table_output_library}')
                raise e
            
            table_info.append({
                'out_library': table_output_library,
                'table_reference': reference,
                'row_count': table.shape[0],
                'column_count': table.shape[1],
            })

        return pd.DataFrame(table_info)


df = reference_parsing(r_local)
df.head()

Save table tbl_0dae0f762483d869bf91feffdd008 to caslib test
Save table tbl_ec49a959e4682ab095086dc8a6988 to caslib test


,out_library,table_reference,row_count,column_count
0,test,tbl_0dae0f762483d869bf91feffdd008,5,5
1,test,tbl_ec49a959e4682ab095086dc8a6988,5,3


In [8]:
r_cloud['tables'][0].get('columnCount')

5

In [40]:
def table_parsing(result) -> pd.DataFrame: #TODO
        tables = result_to_dfs(result)
        select_table = True
        table_selection_idx = 1
        table_selection_method = 'index'

        # select specific table 
        if select_table:
            if table_selection_method.upper() == 'INDEX': # Table with index == table_selection_idx
                parsed_result = tables[table_selection_idx]
            elif table_selection_method.upper() == 'SIZE': # Table with most cells
                table_most_cells = max(tables, key=lambda x: x.size, default=None)
                try:
                    parsed_result = table_most_cells
                except:
                    parsed_result = None

            else:
                raise ValueError(f'Invalid table selection method: {table_selection_method}')

        return parsed_result


df = table_parsing(r_local)
df.head()


,name,birthdate,married
1,Bernd,12.05.92,yes
2,Klara,13.05.92,yes
3,Silvi,14.05.92,yes
4,Cho,15.05.92,yes
5,Clark,16.05.92,yes


In [20]:
r_cloud['tables'][0].keys()

dict_keys(['rowCount', 'columnCount', 'cells', 'boundingRegions', 'spans'])

In [21]:
df.size

25